In [ ]:
%matplotlib inline

#Deep Convolutional Generative Adversarial Networks con Pytorch.

## Objetivo

El propósito de este laboratorio es que el alumno se familiarice con las redes generativas antagónicas (Generative Adversarial Networks, GANs) a través de un ejemplo. Se entrenará una de estas redes GAN para generar nuevos dígitos manuscritos tras enseñarle múltiples instancias reales de los mismos. Este documento pretende dar una explicación exhaustiva de cómo y por qué funcionan estos modelos.

Las GANs son un _framework_ que nos permite enseñar un modelo de _Deep Learning_ a capturar la distribución de unos datos de manera que pueda generar nuevos datos de la misma distribución. Las GANs fueron inventadas por Ian Goodfellow en 2014 y descritas en la referencia [1]. Se componen de dos modelos distintos:

- El generador, que se especializa en producir datos sintéticos similares a las imaǵenes de entrenamiento reales.
- El discriminador, que a partir de una imagen de entrada se especializa en decidir si es real o sintética.

Ambas redes se entrenan de manera conjunta, de modo que el generador intenta vencer al discriminador generando datos sintéticos cada vez más similares a los reales; y el discriminador intenta diferenciar los datos sintéticos de los reales, mejorando continuamente en esta tarea.

La siguiente figura muestra un ejemplo sencillo de red generativa antagónica especializada en la generación de dígitos manuscritos [2].

<img src="http://tsc.uc3m.es/~mmolina/images_gans/gan_example.jpg">


Este tutorial es una adaptación del tutorial disponible en https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html.

## Referencias

- [1] GANs. _Generative Adversarial Nets_. https://papers.nips.cc/paper/5423-generative-adversarial-nets.pdf
- [2] Imagen. https://i.blogs.es/bd2b76/gan_arquitectura/1366_2000.jpg
- [3] DCGANs. _Unsupervised Representation Learning With Deep Convolutional Generative Adversarial Networks_. https://arxiv.org/pdf/1511.06434.pdf
- [4] Celeb-A Faces dataset. http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html
- [5] Celeb-A Faces dataset. https://drive.google.com/drive/folders/0B7EVK8r0v71pTUZsaXdaSnZBZzg
- [6] Ganhacks. Best practices to train GANs. https://github.com/soumith/ganhacks
- [7] GAN — Why it is so hard to train Generative Adversarial Networks! https://medium.com/@jonathan_hui/gan-why-it-is-so-hard-to-train-generative-advisory-networks-819a86b3750b
- [8] MNIST handwritten digits database: http://yann.lecun.com/exdb/mnist/
- [9] CIFAR-10 database: https://www.cs.toronto.edu/~kriz/cifar.html

## Antes de empezar

Antes de empezar, necesita configurar algunas cosas en caso de que vaya a utilizar Google Colab. En particular, necesita subir la carpeta que contiene las imágenes de trabajo y cambiar el directorio de trabajo al correspondiente. Para ello, suba la carpeta con la práctica que va a ejecutar (incluyendo la carpeta images) y ejecute el siguiente código:

In [ ]:
#Descomenta únicamente si quieres ejecutar este código en Google Colab
#from google.colab import drive
#import os, sys
#drive.mount('/content/drive')
#print(os.getcwd())
#os.chdir('/content/drive/My Drive/Colab Notebooks/gans_folder') #Here put the full path to the folder where you have the images

Además, si quiere ejecutar el código con soporte a GPU, en Google Colab vaya a `Entorno de ejecución->Cambiar tipo entorno de ejecución` y seleccione GPU en `acelerador por hardware`.

## Parte 1. Fundamento teórico.

Aunque se recomienda echar un vistazo al artículo sobre GANs [1], en esta sección se van a explicar los conceptos más importantes del algoritmo que son necesarios para el desarrollo de la práctica.

### GANs

Como ya se ha comentado en la introducción, las GANs se componen de un generador que engendra imágenes _fake_ similares a las imágenes de entrenamiento, y un discriminador que observa una imagen, y proporciona como salida si es una imagen real de entrenamiento, o una imagen _fake_ del generador. Durante el entrenamiento, el generador está constantemente tratando de engañar al discriminador mediante de la generación de _fakes_ cada vez mejores, mientras que el discriminador trabajar para convertirse en un mejor detective y clasificar correctamente las imágenes reales y _fakes_. El equilibrio de este juego se alcanza cuando el generador es capaz de producir imágenes sintéticas perfectas (como si vinieran directamente de la distribución de los datos de entrenamiento), y el discriminador proporciona un salida al 50% de confianza independientemente de que la salida del generador sea real o _fake_.  

A continuación se define alguna notación que se va a usar a lo largo del tutorial, comenzando por el discriminador. Sea $x$ una imagen. $D(x)$ es la red discriminadora que proporciona como salida la probabilidad de que $x$ provenga de los datos de entrenamiento en lugar de provenir del generador. Aquí, dado que estamos tratando con imágenes, la entrada a $D(x)$ es una imagen de tamaño CxHxW, por ejemplo, 3x64x64. Intuitivamente, $D(x)$ debe ser __ALTA__ cuando $x$ proviene de los datos de entrenamiento y __BAJA__ cuando $x$ proviene del generador. Por tanto, $D(x)$ se puede considerar como un clasificador binario tradicional desde el punto de vista de su salida.

Para la notación del generador, sea $z$ un vector espacial latente muestreado de una distribución normal estándar. $G(z)$ representa la función del generador que mapea el vector latente $z$ al espacio de datos. El objetivo de $G$ es estimar la distribución de la que provienen los datos de entrenamiento ($p_{data}$) para que pueda genera datos sintéticos con la función estimada ($p_g$). Para más detalles, véase [3].

Por tanto, $D(G(z))$ es la probabilidad (escalar) de que la salida del generador $G$ sea una imagen real. Como se describe en el paper de Goodfellow [1], $D$ y $G$ __se enfrentan en un juego minimax en el cual $D$ intenta maximizar la probabilidad que clasifica correctamente imágenes reales y _fakes_ ($logD(x)$), y $G$ intenta minimizar la probabilidad de que $D$ prediga que sus salidas sean _fake_  ($log(1-D(G(x)))$)__. Extraída del artículo, la función de pérdida GAN es:

\begin{align}\underset{G}{\text{min}} \underset{D}{\text{max}}V(D,G) = \mathbb{E}_{x\sim p_{data}(x)}\big[logD(x)\big] + \mathbb{E}_{z\sim p_{z}(z)}\big[log(1-D(G(z)))\big].\end{align}

En teoría, la solución a este juego minimax es el punto de trabajo donde $p_g = p_{data}$, y el discriminador adivina de manera aleatoria si las entradas al mismo (salidas del generador) son reales o _fakes_. Sin embargo, la teoria de convergencia de las GANs todavía se está estudiando de manera activa y en la realidad los modelos no siempre llegan a entrenarse hasta este punto.

### DCGANs

Una DCGAN es una extensión directa de las GANs descritas anteriormente, exceptuando que una DCGAN utiliza explícitamente capas convolucionales y convolucionales-traspuestas en el discriminador y el generador, respectivamente. Fue descrita por primera vez en el artículo de Radford  _Unsupervised Representation Learning With Deep Convolutional Generative Adversarial Networks_ [3]. El discriminador se realiza a partir de capas convolucionales con _stride_ (que permiten disminuir el tamaño de la imagen de entrada, en lugar de con una función fija como las capas de _pooling_ de la red, con filtros que se aprenden y dependen de las entradas); activaciones LeakyReLU y capas de _batch normalization_. La entrada es una imagen de 3x64x64 y la salida es una probabilidad escalar de que la entrada sea de la distribución de datos reales. El generador está compuesto de convoluciones-traspuestas, activaciones ReLU y capas de _batch normalization_. La entrada es un vector latente $z$, que se extrae de una distribución normal estándar, y la salida es una imagen RGB de tamaño 3x64x64. Las capas de convoluciones traspuestas con _stride_ permiten al vector latente ser transformado en un volumen con las mismas dimensiones que la imagen. En el artículo, los autores también proporcionan algunos consejos sobre como inicializar los optimizadores, calcular las funciones de coste e inicializar los pesos del modelo, que se explicarán a lo largo de la sección de Implementación.

## Parte 2. Implementación.

En primer lugar, se importan los paquetes necesarios y se impone una semilla manual a la aleatoriedad para la reproducibilidad de los resultados.

In [ ]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import cv2
import copy

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# torch uses some non-deterministic algorithms
torch.backends.cudnn.enabled = False

### Entradas

Se definen algunas entradas para la ejecución:

-  **dataroot** - el directorio raíz de la base de datos, que se describe en la siguiente sección.
-  **workers** - el número de hebras para cargar los datos con la clase DataLoader.
-  **batch_size** - el tamaño de _batch_ que utiliza el artículo de DCGAN [3] es 128.
-  **image_size** - el tamaño de las imágenes que se utilizan para entrenamiento. Por defecto, esta implementación usa un tamaño de 64x64. Si se desea usar otro tamaño, las estructuras de D y G se deben modificar. Véase <https://github.com/pytorch/examples/issues/70> para más detalles.
-  **nc** - el número de canales de color de las imágenes de entrada. Para imágenes RGB es 3.
-  **nz** - longitud del vector latente.
-  **ngf** - se relaciona con la profundidad de los mapas de características que se utilizan en el generador.
-  **ndf** - define la profundidad de los mapas de características que se utilizan en el discriminador.
-  **num_epochs** - número de _epochs_ para el entrenamiento de la red. Un entrenamiento extenso probablemente consiga mejores resultados, pero es lento.
-  **lr** - tasa de aprendizaje para el entrenamiento. Según el artículo de Radford [3], debe ser 0.0002. Aquí, se modifica para la generación de dígitos.
-  **beta1** - parámetro beta1 para los optimizadores Adam. Se utiliza 0.5, según [3].
-  **ngpu** - número de GPUs disponibles. Si es 0, el código se ejecuta en CPU. Si es mayor que 0 se ejecutará en ese número de GPUs.

In [ ]:
# Root directory for dataset
dataroot = "mnist"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.005

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

### Base de datos

Para este tutorial se utilizará la base de datos `MNIST` [8]. La base de datos se puede descargar desde Pytorch directamente en el directorio `dataroot`.

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# MNIST database
if not os.path.exists(dataroot):
    os.makedirs(dataroot)

# Download files training.pt and test.pt
dataloader = torch.utils.data.DataLoader(
  dset.MNIST(dataroot, train=True, download=True,
                             transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size,shuffle=True, num_workers=workers)

# Plot some training images
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

### Arquitectura de red

Con los parámetros de entrada y la base de datos preparada, podemos pasar a la definición del _framework_ que se va a utilizar. Se comienza con la estrategia de inicialización de los pesos de la arquitectura, y después se hablará del generador, el discriminador y las funciones de pérdida.

#### Inicialización de los pesos

Los autores especifican en [3] que todos los pesos del modelo se deben inicializar de manera aleatoria de una distribución normal con media 0 y desviación típica 0.02. La función `weights_init` toma un modelo sin inicializar y re-inicializa todas las capas convolucionales, capas de convoluciones traspuestas y _batch normalization_ para cumplir este criterio. Esta función se aplica a todos los modelos inmediatamente despoués de su creación.

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

#### Generador

El generador $G$ está diseñado para mapear el vector espacial latente $z$ al espacio de datos. Como nuestros datos son imágenes, convertir $z$ al espacio de datos significa crear una imagen RGB con el mismo tamaño que las imágenes de entrenamiento (por ejemplo, 3x64x64). En la práctica, esto se consigue a través de una serie de capas de convoluciones traspuestas 2D con *stride*, cada una de ellas emparejada con una capa de normalización de _batch_ 2D y una activación ReLU. La salida del generador se pasa por una función `tanh` para devolver los datos de salida al rango de entrada de $[-1,1]$. Vale la pena notar la existencia de capas de normalización de _batch_ tras las convoluciones traspuestas, esto es una contribución crítica del artículo [3]. Estas capas ayudan al flujo de gradientes durante el entrenamiento. Una imagen del generador [3] con parámetros *nz*=100, *ngf*=128 y *nc*=3 se muestra a continuación (no es el generador que se va a utilizar en nuestra práctica).

<img src="http://tsc.uc3m.es/~mmolina/images_gans/dcgan_generator.png">

Nótese como las entradas que se han definido en la sección de código anterior: *nz*, *ngf* y *nc*, afectan a la arquitectura del generador. El parámetro *nz* es la longitud del vector de entrada, *ngf* se relaciona con la profundidad de los mapas de características a lo largo del generador (hace referencia a la profundidad del último mapa de características) y *nc* es el número de canales en la imagen de salida (3 para imágenes RGB).

In [ ]:
# Generator Code
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

Se instancia el generador y se aplica la función ``weights_init``. Analice el modelo impreso para entender su estructura.

In [ ]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
netG.apply(weights_init)

# Print the model
print(netG)

### Discriminador

El discriminador $D$ es una red de clasificación binaria que toma una imagen como entrada y proporciona como salida una probabilidad escalar de que la imagen de entrada sea real (de manera opuesta a que sea *fake*). Aquí, $D$ toma la imagen de entrada 3x64x64, la procesa a partir de una serie de capas Conv2d, BatchNorm2d y LeakyRelu, y proporciona como salida la probabilidad final a través de una función de activación `Sigmoide`. Esta arquitectura se puede extender con más capas si es necesario, pero es importante el uso del _stride_ mayor que 1 en las convoluciones en lugar de usar capas de *max pooling*. De este modo, la red aprende su propia función de *pooling*. Asimismo, las capas de normalización de _batch_ y las activaciones Leaky proporcionan un flujo de gradiente estable, crítico para el proceso de aprendizaje de $G$ y $D$.

In [ ]:
# Discriminator Code
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

Ahora, como con el generador, se crea el discriminador y se aplica la función ``weights_init``.

In [ ]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
print(netD)

#### Funciones de pérdida y optimizadores

Con $D$ y $G$ definidas, se puede definir el proceso de aprendizaje a través de las funciones de pérdida y optimizadores. Se utilizará la función BCELoss (cross-entropía binaria) tal y como se define en Pytorch (véase https://pytorch.org/docs/stable/nn.html#torch.nn.BCELoss):


\begin{align}\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad l_n = - \left[ y_n \cdot \log x_n + (1 - y_n) \cdot \log (1 - x_n) \right]\end{align}

Nótese como la función proporciona el cálculo de ambas componentes *log* en la función objetivo (por ejemplo, $log(D(x))$ and $log(1-D(G(z)))$). Se puede especificar qué parte de la ecuación BCE utilizar con la entrada $y$. Esto se realiza en el bucle de entrenamiento que se describirá a continuación, pero es importante entender cómo se puede elegir cada componente que se quiere calcular sólo con cambiar $y$ (las etiquetas *ground-truth*).

A continuación, se define la etiqueta real como 1 y la etiqueta _fake_ como 0. Estas etiquetas se utilizarán cuando se calculen las funciones de pérdida de $D$ y $G$, y esta es la convención que se utiliza en el artículo original sobre GANs [1]. Finalmente, se especifican dos optimizadores separados, uno para la red de generación $G$ y otro para la red de discriminación $G$. Como se especifica en [3], ambos son optimizadores Adam con tasa de aprendizaje 0.0002 y Beta1 = 0.5 (para nuestro caso la tasa de aprendizaje es distinta). Para visualizar la progresión del aprendizaje de la red, se genera un _batch_ fijo de vectores latentes que se extraen de una distribución Gaussiana (ruido fijo). En el bucle de entrenamiento, periódicamente se pone como entrada a $G$ este ruido fijo, y a medida que avanza el proceso de aprendizaje se puede ver las imágenes que forma ese ruido.

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

### Entrenamiento

Por último, ahora que se tienen todas las partes del _framework_ GAN definidas, se puede entrenar el mismo. El entrenamiento de las GANs es un proceso __COMPLEJO__, ya que una selección de hiperparámetros incorrecta puede conducir a un no entrenamiento de la misma o al *mode collapse* (es decir, el generador colapsa y solo es capaz de ofrecer un número limitado de *fakes*) sin ninguna explicación válida de por qué ocurre esto. Aquí se seguirá el algoritmo 1 del artículo de Goodfellow [1], acatando el código Ganhacks de buenas prácticas de entrenamiento, del que se dispone en [6]. De este modo, se construirán diferentes *mini-batches* para imágenes reales y *fake*, y se ajustará la función objetivo de $G$ para maximizar $logD(G(z))$. El proceso de entrenamiento se divide en dos partes. La parte 1 actualiza el discriminador y la parte 2 actualiza el generador.

**Parte 1 - Entrenando el Discriminador**

El objetivo del entrenamiento del discriminador es maximizar la probabilidad de clasificar correctamente una imagen de entrada como imagen real o sintética. En palabras de Goodfellow, queremos "actualizar el discriminador haciendo ascender su gradiente estocástico". En la práctica, queremos maximizar $log(D(x)) + log(1-D(G(z)))$. Debido a la sugerencia de *mini-batches* separados de Ganhacks, se calculará esto en dos pasos. Primero, se construye un _batch_ de muestras reales de la base de datos de entrenamiento y se realiza el _forward_ a través de $D$, se calcula la pérdida ($log(D(x))$) y se calculan los gradientes con el método _backward_. Segundo, se construye un _batch_ de muestras sintéticas con el generador, se realiza el _forward_ a través de $D$, se calcula la pérdida ($log(1-D(G(z)))$) y se *acumulan* los gradientes con el _backward_. Una vez se tienen los gradientes acumulados de los _batches_ todo-real y todo-*fake*, se considera que se ha realizado un paso del optimizador del discriminador.

**Parte 2 - Entrenando el Generador**

Como se especifica en el artículo original [1], se quiere entrenar el generador minimizando $log(1-D(G(z)))$ en un esfuerzo por generar mejores *fakes*. Sin embargo, Goodfellow demostró que esto no proporciona suficientes gradientes, especialmente en los primeros pasos de entrenamiento. Como solución, se intenta maximizar $log(D(G(z)))$. En el código esto se consigue: clasificando la salida del generador de la parte 1 con el discriminador, calculando la pérdida de $G$ *usando las etiquetas reales como ground-truth*, computando los gradientes de $G$ con un _backward_ y actualizando los parámetros de $G$ con un paso del optimizador. Puede parece contra-intuitivo usar las etiquetas reales como etiquetas *ground-truth* para la función de pérdida, pero esto nos permite usar la parte $log(x)$ de la BCELoss (en lugar de usar la parte $log(1-x)$), que es exactamente lo que queremos.

Finalmente, se reportan las estadísticas y al final de cada _epoch_ se utilizará el _batch_ de vectores de ruido fijo para visualizar el progreso del aprendizaje de $G$. Las estadísticas que se reportan son:

-  **Loss_D** - pérdida del discriminador calculada como la suma de las pérdidas para los _batches_ todo-real y todo-*fake* ($log(D(x)) + log(D(G(z)))$).
-  **Loss_G** - pérdida del generador calculada como $log(D(G(z)))$.
-  **D(x)** - la salida media (a lo largo del *batch*) del discriminador para el *batch* todo-real. Debe comenzar próxima a 1 y en teoría converger a 0.5 a medida que $G$ va mejorando. Piense por qué ocurre esto.
-  **D(G(z))** - la salida media (a lo largo del *batch*) del discriminador para el *batch* todo-*fake*. El primer número es antes de que $D$ se actualice y el segundo número es después de que $D$ se actualice. Estos números deben comenzar próximos a 0 y converger a 0.5 a medida que $G$ comienza a mejorar. Piense por qué ocurre esto.


****IMPORTANTE**. Este paso puede tardar, dependiendo de cuántos _epochs_ realizar y si se eliminaron algunas muestras de la base de datos.

In [ ]:
# Training Function
def train_gan(netD, netG, dataloader,optimizerD, optimizerG, num_epochs=5,results_dir='results',manualSeed=1):

    # Lists to keep track of progress
    img_list    = []
    G_losses    = []
    D_losses    = []
    iters = 0

    print("Starting Training Loop...")
    # For each epoch
    for epoch in range(num_epochs):
        # For each batch in the dataloader
        for i, data in enumerate(dataloader, 0):

            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            ## Train with all-real batch
            netD.zero_grad()
            # Format batch
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)
            label = torch.full((b_size,), real_label, device=device)
            # Forward pass real batch through D
            output = netD(real_cpu).view(-1)
            # Calculate loss on all-real batch
            errD_real = criterion(output, label)
            # Calculate gradients for D in backward pass
            errD_real.backward()
            D_x = output.mean().item()

            ## Train with all-fake batch
            # Generate batch of latent vectors
            noise = torch.randn(b_size, nz, 1, 1, device=device)
            # Generate fake image batch with G
            fake = netG(noise)
            label.fill_(fake_label)
            # Classify all fake batch with D
            output = netD(fake.detach()).view(-1)
            # Calculate D's loss on the all-fake batch
            errD_fake = criterion(output, label)
            # Calculate the gradients for this batch
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            # Add the gradients from the all-real and all-fake batches
            errD = errD_real + errD_fake
            # Update D
            optimizerD.step()

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for generator cost
            # Since we just updated D, perform another forward pass of all-fake batch through D
            output = netD(fake).view(-1)
            # Calculate G's loss based on this output
            errG = criterion(output, label)
            # Calculate gradients for G
            errG.backward()
            D_G_z2 = output.mean().item()
            # Update G
            optimizerG.step()

            # Output training stats
            if i % 50 == 0:
                print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                      % (epoch, num_epochs, i, len(dataloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

            # Save Losses for plotting later
            G_losses.append(errG.item())
            D_losses.append(errD.item())

            # Check how the generator is doing by saving G's output on fixed_noise
            if (iters % 50 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
                with torch.no_grad():
                    fake = netG(fixed_noise).detach().cpu()
                img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

            iters += 1
        # Save results for each epoch
        if not os.path.exists(results_dir):
            os.makedirs(results_dir)
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu()
        img_grid=np.array(255.0*np.transpose(vutils.make_grid(fake, padding=2, normalize=True),(1,2,0))).astype(np.uint8)
        cv2.imwrite(os.path.join(results_dir,'epoch_'+str(epoch)+'.png'),cv2.cvtColor(img_grid, cv2.COLOR_RGB2BGR))

    # For the purpose of representation, we implement a moving average filter to reduce the noise:
    # consecutive iterations can have very different loss values but we are interested in their trend
    N = 20
    cumsumD, moving_avesD = [0], []
    cumsumG, moving_avesG = [0], []

    for i, x in enumerate(D_losses, 1):
        cumsumD.append(cumsumD[i-1] + x)
        if i>=N:
            moving_aveD = (cumsumD[i] - cumsumD[i-N])/N
            moving_avesD.append(moving_aveD)

    for i, x in enumerate(G_losses, 1):
        cumsumG.append(cumsumG[i-1] + x)
        if i>=N:
            moving_aveG = (cumsumG[i] - cumsumG[i-N])/N
            moving_avesG.append(moving_aveG)
    return (moving_avesD, moving_avesG, img_list)
    #return (D_losses, G_losses, img_list)

## Parte 3. Experimentos.

### 1. Análisis de un entrenamiento correcto.

- Analice cómo las funciones de pérdida de $D$ y $G$ cambian durante el entrenamiento.

****IMPORTANTE**. El valor de learning rate que se os proporciona en la práctica (0.005) es elevado. Es posible que con un valor más pequeño de la tasa de aprendizaje (por ejemplo, 0.001) los resultados sean mejores y las funciones de pérdida evolucionen de manera más suave, pero no está garantizada la convergencia del algoritmo para el caso de los dígitos, es posible que tengáis que hacer varias pruebas.
Tened en cuenta esta información a la hora de realizar vuestra parte de trabajo autónomo, sobre todo si cambiáis de base de datos. Es posible que tengáis que probar diversas tasas de aprendizaje hasta entrenar adecuadamente la red.

In [ ]:
# Training stage
baseline_dir=os.path.join('results','baseline')
if not os.path.exists(baseline_dir):
    os.makedirs(baseline_dir)
[D_losses,G_losses,img_list]=train_gan(netD, netG, dataloader, optimizerD, optimizerG,num_epochs,baseline_dir,manualSeed)
state = {'state_dict': netD.state_dict(),}
torch.save(state, os.path.join(baseline_dir,'netD.pth'))
state = {'state_dict': netG.state_dict(),}
torch.save(state, os.path.join(baseline_dir,'netG.pth'))
np.save(os.path.join(baseline_dir,'G_losses.npy'),np.array(G_losses))
np.save(os.path.join(baseline_dir,'D_losses.npy'),np.array(D_losses))

In [ ]:
# Loss versus training iteration
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

- Visualice las salidas de $G$ para el _batch_ de ruido fijo para cada _epoch_ con una animación.

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

- Observe un _batch_ de datos reales en comparación con un _batch_ de datos *fake*. ¿Son realistas estos últimos?

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

- Se van a generar otros *batches* de imágenes *fake* a partir de muestras de ruido aleatorio. Analice los resultados. ¿Qué dígitos son más reconocibles? ¿Hay algunos que no se generan correctamente?

In [ ]:
plt.figure(figsize=(15,15))

for i in range(4):
    with torch.no_grad():
        random_noise = torch.randn(64, nz, 1, 1, device=device)
        fake = netG(random_noise).detach().cpu()
    plt.subplot(2,2,i+1)
    plt.axis("off")
    plt.imshow(np.transpose(vutils.make_grid(fake[:64], padding=5, normalize=True).cpu(),(1,2,0)))
plt.show()

- ¿Qué ocurre si se ponen a 0 o a un valor fijo algunas de las entradas de ruido blanco del generador? ¿Para cada valor de entrada, la red proporciona un solo modo o varios modos distintos?

In [ ]:
# We put some dimensions to 0
random_noise = torch.randn(64, nz, 1, 1, device=device)
# We put random elements to 0
for i in range(20):
    random_noise[random.randint(0,63),:,:,:]=0
with torch.no_grad():
    fake = netG(random_noise).detach().cpu()
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(fake[:64], padding=5, normalize=True).cpu(),(1,2,0)))
# We fix a lot of elements with the same value
random_noise = torch.randn(64, nz, 1, 1, device=device)
for i in range(20):
    random_noise[random.randint(0,63),:,:,:]=-1
with torch.no_grad():
    fake = netG(random_noise).detach().cpu()
plt.subplot(1,2,2)
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(fake[:64], padding=5, normalize=True).cpu(),(1,2,0)))
plt.show()

- En GANs no se calcula específicamente una representación latente a partir de los datos reales, con lo que no se puede hacer directamente la mezcla de dos muestras de ruido de entrada para producir una señal de salida mezcla de ambas. Sin embargo, si se toman dos muestras de ruido y su suma, ¿la imagen resultante es una mezcla de las otras dos imágenes? ¿Contiene componentes de ambas imágenes?

In [ ]:
random_noise = torch.randn(64, nz, 1, 1, device=device)
# We change the sample to the structure: sample1-sample2-sum2//sample1-sample2-sum//...
for i in range (2,64,3):
    random_noise[i,:,:,:]=random_noise[i-1,:,:,:]+random_noise[i-2,:,:,:]

with torch.no_grad():
    fake = netG(random_noise).detach().cpu()
plt.figure(figsize=(15,15))
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(fake[:60], nrow=3,padding=5, normalize=True).cpu(),(1,2,0)))
plt.show()

### 2. Problems with GANs.

#### _Mode collapse_ [7]

El _mode collapse_ es uno de los problemas más difíciles de resolver con GANs. Las distribuciones de datos reales son multi-modales. Por ejemplo, en la base de datos MNIST se tienen 10 clases (dígitos manuscritos de 0 a 9). El _mode collapse_ hace referencia a un problema de entrenamiento de las GANs según el cual la red generativa sólo es capaz de producir uno de los modos de la distribución, véase la figura siguiente [7], o una mezcla de los modos de la distribución. La figura hace referencia a dos GANs entrenadas para generar dígitos manuscritos a partir de MNIST. Mientras que la primera está correctamente entrenada y es capaz de producir imágenes sintéticas que comprenden todos los dígitos, la segunda sólo es capaz de producir el dígito 6.

<img src="http://tsc.uc3m.es/~mmolina/images_gans/mode_collapse.png">

Este problema de entrenamiento es bastante común, como también es usual que se produzca un *partial collapse*, es decir, que la red produzca imágenes sintéticas que responden a pocos modos. Un ejemplo para la generación de caras se puede observar en la figura de abajo. Muchos de los rostros sintéticos que genera la red son muy similares. Si se continúa entrenando, estos modos dominarán y las imágenes que genere la red responderán solo a dichos modos.

<img src="http://tsc.uc3m.es/~mmolina/images_gans/caras_collapse.png">


#### _Diminished gradient_ [7]

Otro de los problemas principales que se producen a la hora de entrenar una GAN es el _diminished gradient_. Esto ocurre cuando el discriminador es tan bueno que el gradiente en el generador se desvanece y no se aprende nada, es decir, el generador no se actualiza de manera suficiente como para proporcionar buenos _fakes_.

- ¿Cómo podría simular este problema para el escenario que se está modelando?

Aparte de la no convergencia, el *mode collapse* y el *diminished gradient* son los problemas más usuales al entrenar GANs. Para un análisis más detallado de la convergencia de las redes generativas, véase [7].

### 3. Uso del *batchnorm*

En este apartado se va a justificar experimentalmente la decisión de diseño de las redes generativas de utilizar la normalización de *batch*. Para ello se generan:

- Un generador y un discriminador en los que se elimina el *batch normalization*.

Se repite el mismo experimento y se analiza el resultado.


In [ ]:
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.backends.cudnn.enabled = False
# No batchnorm
batchnorm_dir=os.path.join('results','no_batchnorm')
if not os.path.exists(batchnorm_dir):
    os.makedirs(batchnorm_dir)

class Generator_no_batchnorm(nn.Module):
    def __init__(self, ngpu):
        super(Generator_no_batchnorm, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

class Discriminator_no_batchnorm(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator_no_batchnorm, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

# Generator without batchnorm
netG_no_batchnorm = Generator_no_batchnorm(ngpu).to(device)
if (device.type == 'cuda') and (ngpu > 1):
    netG_no_batchnorm = nn.DataParallel(netG_no_batchnorm, list(range(ngpu)))
netG_no_batchnorm.apply(weights_init)
netG.apply(weights_init)

# Discriminator without batchnorm
netD_no_batchnorm = Discriminator_no_batchnorm(ngpu).to(device)
if (device.type == 'cuda') and (ngpu > 1):
    netD_no_batchnorm = nn.DataParallel(netD_no_batchnorm, list(range(ngpu)))
netD_no_batchnorm.apply(weights_init)

# Learning rate for optimizers
lr_D = lr
lr_G = lr

# Setup Adam optimizers for both G and D
optimizerD_no_batchnorm = optim.Adam(netD_no_batchnorm.parameters(), lr=lr_D, betas=(beta1, 0.999))
optimizerG_no_batchnorm = optim.Adam(netG_no_batchnorm.parameters(), lr=lr_G, betas=(beta1, 0.999))
[D_losses_no_batchnorm,G_losses_no_batchnorm,img_list_no_batchnorm]=train_gan(netD_no_batchnorm, netG_no_batchnorm, dataloader, optimizerD_no_batchnorm, optimizerG_no_batchnorm,num_epochs,batchnorm_dir)

# Plot the fake images from the last epoch
plt.figure(figsize=(7,7))
plt.axis("off")
plt.title("No batchnorm")
plt.imshow(np.transpose(img_list_no_batchnorm[-1],(1,2,0)))
plt.show()

### 4. Evaluación del trabajo autónomo del alumno.

#### Criterios de evaluación

De esta práctica (si elegida) surge la segunda evaluación para la asignatura. Una vez comprendidos los fundamentos de las GANs, puede realizar los experimentos que considere oportunos. Estos experimentos pueden ir dirigidos a:

- Profundizar en la arquitectura (observar la dependencia de los resultados con las modificaciones de la misma).
- Analizar los resultados y proporcionar estrategias de mejora.
- Aplicar el modelo a otras tareas, analizando las diferencias con la tarea propuesta.


#### Entregables

- Presentación (Fecha indicada en la entrega del proyecto en Aula Global). Este día cada grupo de alumnos tendrá un turno de 10 minutos de preguntas (máximo 5 minutos de presentación) sobre el apartado de trabajo autónomo con ayuda de un máximo de 3 transparencias.
- Informe + Código. Los alumnos entregarán un breve informe (2 caras para la descripción, 1 cara de referencias y figuras si fuese necesaria) donde describirán los aspectos más importantes de la solución propuesta. El objetivo es que el alumno describa los análisis y extensiones que ha planteado al modelo y justifique su objetivo y utilidad de manera breve. Asimismo, se proporcionará el código utilizado para los experimentos (bien sobre este mismo Notebook, en formato `.ipynb` o bien en código Python, en formato `.py`).

La fecha límite de entrega del fichero de código y el informe es la fecha indicada en la entrega del proyecto en Aula Global.


#### Sugerencias

A continuación se proporcionan algunas sugerencias para que el alumno trabaje de manera autónoma, a título informativo. Si lo desea, puede centrarse en implementar una o varias de ellas.

- Entrene durante más tiempo para ver si mejoran los resultados. Analice los resultados menos realistas y proponga estrategias de mejora.

- Modifique este modelo: cambie el tamaño de las imágenes y la arquitectura. Analice los resultados.

- Modifique y entrene la red en otras bases de datos, analice las diferencias con el caso de la generación de dígitos y entre ellas:
  - `Celeb-A Faces`: para descargar la base de datos `Celeb-A Faces`, utilizar el link de [4] o `Google Drive` [5]. La base de datos se descargará como un archivo llamado *img_align_celeba.zip*. Una vez descargada, genere un directorio llamado *celeba* y extraíga el archivo .zip en ese directorio. La estructura del directorio resultante debe ser `/path/to/celeba-> img_align_celeba  -> 188242.jpg 173822.jpg ...` Este es un paso importante ya que utilizaremos la clase ImageFolder para crear el dataset en Pytorch, que requiere que haya subdirectorios en la carpeta raíz de la base de datos. Analice la mezcla de imágenes a la salida de la red en este caso.
  - CIFAR-10: para descargar la base de datos de CIFAR-10 se utiliza la clase de Pytorch.
  
  Se incluye un ejemplo de código de la celda de abajo para la carga de ambas bases de datos.

- Si entrena la red en una base de datos que permite clasificación (por ejemplo, MNIST o CIFAR-10), se pueden utilizar las características extraídas por el discriminador para construir un clasificador sencillo, como en [3].

In [ ]:
# # Code example for Celeba and CIFAR-10

# # CELEBA
# dataroot = "celeba"
# # We can use an image folder dataset the way we have it setup.
# # Create the dataset
# dataset = dset.ImageFolder(root=dataroot,
#                            transform=transforms.Compose([
#                                transforms.Resize(image_size),
#                                transforms.CenterCrop(image_size),
#                                transforms.ToTensor(),
#                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                            ]))
# # Create the dataloader
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                          shuffle=True, num_workers=workers)

# # Plot some training images
# real_batch = next(iter(dataloader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# # CIFAR10
# dataroot = "cifar10"
# if not os.path.exists(dataroot):
#     os.makedirs(dataroot)

# dataset = dset.CIFAR10(dataroot, train=True, download=True,
#                            transform=transforms.Compose([
#                                transforms.Resize(image_size),
#                                transforms.ToTensor(),
#                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
#                            ]))
# # Create the dataloader
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                          shuffle=True, num_workers=workers)

# # Plot some training images
# real_batch = next(iter(dataloader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))
